# Homework 2: Naive Bayes
## Due September 19th

## Legal reasoning (From Murphy, 2.2). (25 pts)

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.  The defendant is known to have this rare blood type.  The  prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy. What is wrong with this argument?

### Answer

The Probability that the defendent has a given blood type is .01. We also have the information that whoever commited the crime also has this blood type. So the attorney needs to find the probability that the defendent is guilty given that their blood type was found at the crime scene.

From Bayes Theorem, 
$$P(\mathbf{w}|y) = \frac{P(y|\mathbf{w})P(\mathbf{w})}{P(y)},$$

w innocence, and y is the blood type. $P(y) = .01$, that is pretty straight forward, one percent of people have this blood type. For $P(y|w)$ the prosecutor has assumed this to be 1, since there is compelling evidence that a person with this blood type commited the crime. In other words the probability that the defendent has the blood type given that they are guilty is 1. The last number, is the probability that a person is innocent of the crime independent of the test $P(w)$. This number is not well constrained. Plugging in the numbers above and solving for $P(w)$ we find that $P(w)=.0001$ for there to be a 1% chance of innocence given the blood type. This seems quite low all things considered. Essentially the prosecutor has presupposed guilt in making this argument.

## Ham vs Spam (75 pts)
One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|X=x) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|X=x) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [22]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()
# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

Below, I have provided code skeletons. Your job is to make the code skeletons into an operational naive Bayes spam detector. (you may discard these skeletons if you would prefer to code this from scratch). Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:

In [38]:
#What is the prior P(Y=ham) ?
ham_prior =  np.sum(training_labels)/float(len(training_labels)) # The sum of the observed hams, 
#divided by the total number of observations
totalham = 0
totalspam = 0
for key,val in word_dictionary.items(): #finding the total number of spam and ham words
    totalham += val[1]
    totalspam += val[0]
print totalham, totalspam
# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
spam_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = float(val[1])/(totalham) #!The mentions in hams, total number of words in ham
    spam_likelihood[key] = float(val[0])/(totalspam) #same but with spam values
print ham_likelihood['cuz'], spam_likelihood['cuz']

67185 25750
8.93056485823e-05 3.88349514563e-05


Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [39]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))
# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = 1.0*ham_prior 
    posterior_spam = 1.0*(1.-ham_prior)
    #! Don't forget to include the prior!
    # Loop over all the words in each message
    for w in m:
        # #! What is the purpose of this try/except handler?
        # If a word is passed that is not in the dictionary the loop will break,
        #so there needs to be a way to avoid words that don't show up in the training set
        try:
            posterior_ham *= ham_likelihood[w]#! multiply the ham prior by all the probability of a word in ham
            posterior_spam *= spam_likelihood[w] #! same but with spam
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in double_scalars
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: RuntimeWarning: invalid value encountered in double_scalars


Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [40]:
#print posteriors, test_labels
post2 = posteriors
post2[(post2>.5)]=1. #converting to boolean arrays, if prob spam > prob ham, it's spam
post2[(post2<.5)]=0.
post2 = post2[:,1] #the second column is the 0-spam 1-ham column
#!

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


In [41]:
trials = float(len(post2)) #number of comparisons in test set
total = post2+test_labels #Adding my results to the test labels if the sum is 1,
#then my results differ from the test and are incorrect. spam+spam = 0, ham+ham=2, ham+spam=1
totalwrong = len(total[total==1.]) # the total number of incorrect responses
print 'Total wrong =', totalwrong
print 'Number of trials =', trials
percentright=(1.-totalwrong/trials)*100 #the percent correct is just 1- the incorrect percent
print 'My filter is '+str(percentright)+'% accurate.'  #I'm getting a better accuracy with this method

Total wrong = 14
Number of trials = 1114.0
My filter is 98.7432675045% accurate.
